# scRNA-seq Data Embedding with PCA

 This notebook demonstrates how to:
 1. Load AnnData objects for scGeneScope round_1 and round_2 h5ad datasets
 2. Compute PCA on the train subset of the round_1 data (based on replicate 3)
 3. Project both round_1 and round_2 data using the learned PCA transformation
 4. Train a Ridge classifier on round_1 replicate 3 and evaluate on held-out replicates
 5. Test generalization by evaluating on the round_2 dataset

In order to run this notebook, make sure you have downloaded the scGeneScope round_1.h5ad and round_2.h5ad according to the instructions in the README.md

 NOTICE: The scGeneScope dataset is large and running this notebook requires a high memory (ideally ~512G RAM) instance.

## Setup and Imports

In [10]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Union, Tuple, List, Dict

import anndata as ad
from sklearn.decomposition import PCA

# Set random seed for reproducibility
np.random.seed(42)

## Helper Functions

In [3]:
def load_anndata(filepath: str) -> ad.AnnData:
    """
    Load an AnnData object from a file.
    
    Args:
        filepath: Path to the AnnData file (.h5ad)
        
    Returns:
        Loaded AnnData object
    """
    print(f"Loading AnnData from {filepath}")
    return ad.read_h5ad(filepath)

def get_subset_by_obs(
    adata: ad.AnnData, 
    obs_key: str, 
    obs_values: Union[str, List[str]]
) -> ad.AnnData:
    """
    Extract a subset of cells from AnnData based on observation values.
    
    Args:
        adata: AnnData object
        obs_key: Column name in adata.obs to filter on
        obs_values: Value(s) to include from the obs_key column
        
    Returns:
        Subset of the original AnnData
    """
    if isinstance(obs_values, str):
        obs_values = [obs_values]
    
    mask = adata.obs[obs_key].isin(obs_values)
    subset = adata[mask].copy()
    
    print(f"Created subset with {subset.n_obs} cells based on {obs_key}")
    return subset

def compute_pca(
    adata_subset: ad.AnnData, 
    n_components: int = 50
) -> Tuple[PCA, np.ndarray]:
    """
    Fit PCA on a training subset (e.g. replicate 3) using sklearn's implementation.
    
    Args:
        adata_subset: AnnData subset from the training replicate
        n_components: Number of PCA components to compute
    
    Returns:
        A tuple of:
        - Fitted PCA model
        - The PCA embeddings (transformed data)
    """
    print("Using .X for PCA (data should be normalized log1p transformed)")
    
    # Get the data matrix
    # Filter to only include treatment group
    treatment_mask = adata_subset.obs['Group'] == 'treatment'
    adata_subset_treatment = adata_subset[treatment_mask].copy()
    X = adata_subset_treatment.X.copy()
    print(f"Filtered to {adata_subset_treatment.n_obs} cells in treatment group")
    if not isinstance(X, np.ndarray):
        X = X.toarray()
    
    # Initialize and fit the PCA model
    pca_model = PCA(
        n_components=n_components,
        random_state=42,
    )
    
    # Fit and transform the data
    X_pca = pca_model.fit_transform(X)
    
    print(f"Computed PCA with {n_components} components")
    print(f"Explained variance ratio: {np.sum(pca_model.explained_variance_ratio_):.2f}")
    
    return pca_model, X_pca

def project_all_data(
    adata: ad.AnnData, 
    pca_model: PCA,
    obsm_key: str = 'PCA',
    debug: bool = False
) -> ad.AnnData:
    """
    Project the full AnnData using a fitted sklearn PCA model.
    
    Args:
        adata: Full AnnData object with all cells
        pca_model: Fitted sklearn PCA model
        obsm_key: Key to store projected embeddings in adata.obsm
        debug: If True, print additional diagnostic information
    
    Returns:
        Updated AnnData with projected embeddings
    """
    print("Using .X for projection")
    
    # Get the data matrix
    X = adata.X
    if not isinstance(X, np.ndarray):
        X = X.toarray()
    
    if debug:
        print(f"Data shape: {X.shape}")
        print(f"PCA model components shape: {pca_model.components_.shape}")
        print(f"First few feature names: {list(adata.var_names[:5])}")
        print(f"Data statistics - Mean: {np.mean(X):.4f}, Std: {np.std(X):.4f}, Min: {np.min(X):.4f}, Max: {np.max(X):.4f}")
    
    # Transform the data using the fitted PCA model
    X_proj = pca_model.transform(X)
    
    # Store the projected embeddings
    adata.obsm[obsm_key] = X_proj
    
    print(f"Projected {adata.n_obs} cells using PCA from training replicate")
    
    return adata

In [4]:
# Load the round_1 data
round_1_adata = load_anndata("../data/measured/rnaseq/round_1.h5ad")

Loading AnnData from ../data/measured/rnaseq/round_1.h5ad


In [5]:
# Parameters -- fit PCA on replicate 3 (treated as training set) and project all data
obs_key = 'Replicate'    # Key in APP_adata.obs to subset on
obs_value = '3'          # Value to filter for
n_pca_components = 2000  # Number of PCA components to compute

round_1_adata_train = get_subset_by_obs(round_1_adata, obs_key, obs_value)

# Compute PCA on the training replicate
pca_model, X_pca_subset = compute_pca(
    round_1_adata_train, 
    n_components=n_pca_components
)

Created subset with 115308 cells based on Replicate
Using .X for PCA (data should be normalized log1p transformed)
Filtered to 103865 cells in treatment group
Computed PCA with 2000 components
Explained variance ratio: 0.50


In [6]:
# Project all cells using the PCA parameters computed from the training replicate
round_1_adata = project_all_data(
    round_1_adata,
    pca_model,
    obsm_key='PCA'
)

Using .X for projection
Projected 406412 cells using PCA from training replicate


In [7]:
# Load the round_2 data
round_2_adata = load_anndata("../data/measured/rnaseq/round_2.h5ad")

# Display basic information about the loaded data
print(f"Loaded round_2 AnnData with {round_2_adata.n_obs} cells and {round_2_adata.n_vars} genes")
print(f"Available observation keys: {list(round_2_adata.obs.columns)}")
print(f"Available obsm keys: {list(round_2_adata.obsm.keys())}")

Loading AnnData from ../data/measured/rnaseq/round_2.h5ad
Loaded round_2 AnnData with 221292 cells and 55308 genes
Available observation keys: ['Sample_ID', 'percent.mt', 'postqc_cells', 'RNA_snn_res.0.8', 'nFeature_RNA', 'Phase', 'Treatment', 'batch', 'preqc_cells', 'nCount_RNA', 'G2M.Score', 'S.Score', 'seurat_clusters', 'orig.ident', 'source', 'Replicate', 'Seen', 'Group']
Available obsm keys: []


In [11]:
# Define output paths for saving the processed AnnData objects
round_1_output_path = "../data/features/rnaseq/pca/n2000/round_1.h5ad"
round_2_output_path = "../data/features/rnaseq/pca/n2000/round_2.h5ad"

# Create output directories if they don't exist
os.makedirs(os.path.dirname(round_1_output_path), exist_ok=True)
os.makedirs(os.path.dirname(round_2_output_path), exist_ok=True)

# Save round_1 (round_1_adata) to disk
print(f"Saving round_1 dataset with PCA embeddings to: {round_1_output_path}")
# Convert X and obsm to float32 to reduce memory usage
round_1_adata.X = round_1_adata.X.astype('float32')
for key in round_1_adata.obsm.keys():
    round_1_adata.obsm[key] = round_1_adata.obsm[key].astype('float32')

round_1_adata.write_h5ad(round_1_output_path)

# Save round_2 (AP_adata) to disk
print(f"Saving round_2 dataset with PCA embeddings to: {round_2_output_path}")
# Convert X and obsm to float32 to reduce memory usage
round_2_adata.X = round_2_adata.X.astype('float32')
for key in round_2_adata.obsm.keys():
    round_2_adata.obsm[key] = round_2_adata.obsm[key].astype('float32')

round_2_adata.write_h5ad(round_2_output_path)

print("Both datasets successfully saved with PCA embeddings.")

Saving round_1 dataset with PCA embeddings to: ../data/features/rnaseq/pca/n2000/round_1.h5ad
Saving round_2 dataset with PCA embeddings to: ../data/features/rnaseq/pca/n2000/round_2.h5ad
Both datasets successfully saved with PCA embeddings.


PCA embeddings are now saved and ready to be used in the scGeneScope library. 

Other embeddings for foundation models are created according to the respective model documentation and examples, ie see:
- [scGPT](https://github.com/bowang-lab/scGPT/blob/0cd3c73779e93e999789d52b4412e6c23baaa02b/scgpt/tasks/cell_emb.py#L148)
- [scVI1/2](https://chanzuckerberg.github.io/cellxgene-census/notebooks/analysis_demo/comp_bio_scvi_model_use.html)
- [geneformer](https://chanzuckerberg.github.io/cellxgene-census/notebooks/analysis_demo/comp_bio_geneformer_prediction.html)
- [UCE](https://github.com/snap-stanford/UCE)